In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
plotwidth=40

In [ ]:
import pandas as pd
import math
from natsort import natsort_keygen
import glob
import yaml
import numpy as np
import datetime
import pandas as pd
import math
from natsort import natsort_keygen
import glob
import yaml
import numpy as np
import datetime
import statsmodels.api as sm
from tqdm.notebook import tqdm, trange
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression
from scipy.optimize import nnls
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
import json
import psycopg2
import netrc
import re
import os
from multiprocess import Pool

lowess = sm.nonparametric.lowess

# Globals

In [ ]:
datadir = '.'
plotdir = 'plots'
vpipe_working = 'working' # V-pipe's working directory


tally_data = os.path.join(datadir, 'tallymut_line.tsv')
plant_name_tsv = pd.read_csv('ww_plants.tsv')
heatmaps_json_file = os.path.join(datadir, 'ww_update_data_heatmap.json')

#for later
# cooc_data = 'data/ww-cooc.csv'

start_date = '2020-12-08'
todaydate = datetime.date.today().strftime("%Y-%m-%d")
cities_list=['Basel (catchment area ARA Basel)', 'Altenrhein (SG)', 'Chur (GR)', 'Genève (GE)', 'Laupen (BE)',
       'Lausanne (VD)', 'Lugano (TI)', 'Zürich (ZH)', 'Kanton Zürich', ]
cities_list_upload=cities_list #
variants_pangolin={'al':'B.1.1.7','be':'B.1.351','ga':'P.1','C36':'C.36.3','ka':'B.1.617.1','de':'B.1.617.2','AY42':'AY.4.2','B16173':'B.1.617.3','om':'B.1.1.529','om2':'omi-BA.2','BA1':'BA.1','BA2':'BA.2'}

# variants that will be displayed here AND uploaded:
variants_list_upload=['B.1.1.7', 'B.1.351', 'P.1', 'B.1.617.2', 'B.1.617.1', 'B.1.1.529','omi-BA.2',]
# extra variants that will be ONLY display here, but NOT uploaded (typically for variant not present yet)
variants_list = variants_list_upload#+['']
variants_upload = ['undetermined'] + variants_list_upload
variants_not_reported = ['BA.1', 'BA.2', 'C.36.3', 'B.1.617.3', 'AY.4.2','mu','d614g']
rename_variants = { 'omi-BA.2': 'BA.2'}
exclusive_list=['B.1.351','P.1'] # list of variants where we should apply filtering
exclude_from=['B.1.1.7','B.1.351','P.1' ] #,'IN2'] #,'C36','IN1','IN2','IN3'] # filter against these variants
max_pool = len(cities_list)

# mutations type to be considered
mut_types = ['mut', 'extra']
# drop all shared and subset mutations
to_drop = ['subset', 'shared']


# Outputs
plots_dir='deconv_plots'
if not os.path.isdir(plots_dir):
    try:
        os.mkdir(plots_dir, mode=0o775)
    except FileExistsError:
        pass
update_data_combined_file = os.path.join(datadir, 'ww_update_data_combined.json')

# Load Data

In [ ]:
# load tsv into dataframe
df_tally = pd.read_csv(tally_data, sep='\t')
df_tally = df_tally.rename(columns=variants_pangolin)
df_tally = df_tally.drop(variants_not_reported, axis=1)

df_tally.head()
#df_tally[(df_tally['plantname'] == 'Altenrhein (SG)') & (df_tally['date'] == '2021-12-26') & 
#((df_tally['B.1.1.529'] == 'mut') | (df_tally['B.1.1.529'] == 'extra'))].to_csv('Kanton_Basel_mut.csv')
df_tally[(df_tally['plantname'] == 'Altenrhein (SG)') & (df_tally['date'] == '2021-12-26') & (df_tally['B.1.1.529'] == 'mut') & (df_tally['pos'] == 22578) ]

In [ ]:
df_data = df_tally.copy()
df_data.dropna(subset = ["frac", "date"], inplace=True)
df_data['mutations'] = df_data['pos'].astype(str) + df_data['base']

df_data = df_data[~(df_data['base'] == '-') & (df_data['date'] >= start_date)]

df_data = df_data[df_data.columns.difference(['pos', 'gene', 'base'], sort=False)]

print(df_data.shape)
# drop other mutation type from df_data
for v in variants_list:
    df_data = df_data[~df_data[v].isin(to_drop)]

df_data = df_data.reset_index(drop=True)

mutations =  sorted(list(df_data['mutations'].unique()), key=natsort_keygen())
# df_data[df_data['plantname'] == 'Altenrhein (SG)'].sort_values(by='date')

# remove problematic mutations
df_data = df_data[~df_data["mutations"].isin(["28461G", "11201G", "26801C"])]
df_data.head()
df_data[df_data['batch'] == '20220110_HKLFGDRXY']['plantname'].unique()

In [ ]:
df_data2 = df_data[df_data.columns.difference(['batch'], sort=False)]
df_data2 = df_data2.replace(np.nan, 0)
df_data2 = df_data2.replace(['extra', 'mut'], 1)
df_data2 = df_data2[df_data2.columns.difference(['plantcode', 'cov', 'var'], sort=False)]
df_data2 = df_data2.sort_values(by=['date', 'sample'])

dates = sorted(set(df_data2['date']), key=natsort_keygen())
# df_data2[(df_data2['plantname'] == 'Lugano (TI)') & (df_data2['date'] == '2021-07-20')]
df_data2.insert(4, 'undetermined', 0)
# sorted(df_data2['date'].unique())

In [ ]:
#Complement of matrix A (to add undetermined case)
data = {'sample':df_data2['sample'], 'date':df_data2['date'], 'plantname':df_data2['plantname'],
       'frac':1-df_data2['frac']}
data.update({v: 1-df_data2[v] for v in variants_list})
data.update({'mutations': '-' + df_data2['mutations'].astype(str)})

df_data3 = pd.DataFrame(data)
df_data3.insert(4, 'undetermined', 1)
df_data3.head()

In [ ]:
df_data4 = pd.concat([df_data2, df_data3], sort=False)
df_data4[df_data4['undetermined'] == 0]
mutations =  sorted(list(df_data4['mutations'].unique()), key=natsort_keygen())
df_data4[df_data4['B.1.617.2'] == 1]
variants = ['undetermined' ] + variants_list
variants = sorted(variants)#, key=natsort_keygen())
# sorted(df_data4[df_data4['plantname'] == cities_list[2]]['date'].unique())
df_data4.head()
df_data4.shape
# df_data4[(df_data4['plantname'] == 'Basel (catchment area ARA Basel)') & (df_data4['mutations'] == '22578A')]

### Filter out problematic mutations for Omicron in V4/V3 (very temp!)

In [ ]:
df_data4["pos"] = df_data4.mutations.str.extract("([0-9]+)").astype("int")
df_data4[pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")]
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos >= 22428) & (df_data4.pos <= 22785))] # amplicon75
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos >= 22677) & (df_data4.pos <= 23028))] # amplicon76
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos >= 22974) & (df_data4.pos <= 23327))] # amplicon77
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos >= 26277) & (df_data4.pos <= 26635))] # amplicon88
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos >= 26895) & (df_data4.pos <= 27256))] # amplicon90
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos == 26709))] # other
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos == 27807))] # other
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos == 2832))] # other
df_data4 = df_data4[~((pd.to_datetime(df_data4.date) > np.datetime64("2021-11-20")) & (df_data4.pos == 10449))] # other


df_data4.shape


In [ ]:
df_data4 = df_data4.drop("pos", axis=1)

In [ ]:
# df_data4[(df_data4['plantname'] == 'Basel (catchment area ARA Basel)') & (df_data4['mutations'] == '22674T')]

# Ridge Regression

In [ ]:
def ineq_cons(x):
    """constrain all elements of x to be >= 0"""
    return x

def eq_cons(x):
    """constrain the sum of all rows to be equal to 1"""
    return np.sum(x) - 1

def fn(x, A, b):
    return 0.5*np.linalg.norm(A.dot(x)-b)**2

def cal_nnls_ridge2(X,y, lam, l=9):
    p = X.shape[1]
    Xext = np.vstack((X, lam * np.identity(X.shape[1])))

    yext = np.hstack((y, np.zeros(p)))
    coefs, _ = nnls(Xext, yext)

    cons = [{'type': 'ineq', 'fun': ineq_cons},
               {'type': 'eq', 'fun': eq_cons}]

    #Call minimisation subject to these values
    minout = minimize(fn, coefs, args=(Xext, yext), method='SLSQP',bounds=[(0., None) for i in range(l)]
                      ,constraints=cons)

    x = minout.x

    return x    

### Bootstrapping

In [ ]:
def resample_mutations(df_city1, mutations):
    """
    Function to resample mutations by replacement (preserving mutation-complement pairs). 
    Returns a copy of the DataFrame with <resample_value> column indicating how many times the mutation was in the resample.
    """

    # resample indices of mutations with replacement (warning: high is one above actual high!
    rand_idcs = np.random.randint(0, high=int(len(mutations)/2), size=int(len(mutations)/2))
    # for all mutations, count how many times they appear in the resample (0, 1, 2 ...)
    resamples_counts = np.bincount(rand_idcs, minlength=int(len(mutations)/2))
    # make a dict of {mutation : occurences in the resample} pairs
    resample_coeff_dict = dict(zip(mutations, np.concatenate([resamples_counts, resamples_counts])))
    # make a column with coefficients for how many times a row should be accounted for according to the resample
    df_sampled = df_city1.copy()
    df_sampled.loc[:,"resample_value"] = df_sampled.mutations.map(resample_coeff_dict)
    
    return df_sampled, rand_idcs

In [ ]:
# def regress_city(city): # globals: df_data2, df_data4, variants
#     current_ridge = []
# #     df_city1 = df_data2[(df_data2['plantname'] == city)]
#     df_city1 = df_data4[(df_data4['plantname'] == city)]
#     dates = sorted(set(df_city1['date']), key=natsort_keygen())

#     for n in trange(0,100, desc=city):
#         for d in dates:
#             muts = np.random.choice(mutations, len(mutations), replace=True)
#             df_city_date = df_city1[df_city1['date'] == d]


#             df_sampled = df_city_date[df_city_date['mutations'].map(lambda x: x in muts)]
            
# #             rand_idcs = np.random.randint(0, high=int(len(mutations)/2 -1), size=int(len(mutations)/2 -1))
# #             muts_sampled = np.array(mutations)[np.concatenate([rand_idcs, rand_idcs+int(len(mutations)/2)])]
            
# #             df_sampled = df_city_date[df_city_date.mutations.isin(muts_sampled)]
# #             print(df_sampled)



#             if len(df_sampled) > 0:
#                 df_ridge = df_sampled[df_sampled.columns.difference(['sample', 'date', 'plantname', 'mutations'], 
#                                                                     sort=False)]

#                 char_vars = df_ridge[df_ridge.columns.difference(['sample', 'date','plantname',
#                                                                           'frac', 'mutations', 'date'])].columns.values

#                 for value, variant in zip(cal_nnls_ridge2(np.array(df_ridge[variants]), np.array(df_ridge['frac']), 
#                                                           0.5, len(char_vars)), char_vars):
#                         current_ridge.append([variant, value, city, d, n])
#     return current_ridge

In [ ]:
def regress_city(city): # globals: df_data2, df_data4, variants
    current_ridge = []
#     df_city1 = df_data2[(df_data2['plantname'] == city)]
    df_city1 = df_data4[(df_data4['plantname'] == city)]
    dates = sorted(set(df_city1['date']), key=natsort_keygen())
#     print(variants)

    for n in trange(0,100, desc=city):
        for d in dates:
            muts = np.random.choice(mutations, len(mutations), replace=True)
            df_city_date = df_city1[df_city1['date'] == d]


#             df_sampled = df_city_date[df_city_date['mutations'].map(lambda x: x in muts)]
            
            df_sampled,_ = resample_mutations(df_city_date, mutations)

            if len(df_sampled) > 0:
                df_ridge = df_sampled[df_sampled.columns.difference(['sample', 'date', 'plantname', 
                                                                     'mutations', 'resample_value'], sort=False)]

                char_vars = df_ridge[df_ridge.columns.difference(['sample', 'date','plantname', 'frac', 
                                                                  'mutations', 'resample_value' ,'date'])].columns.values

                for value, variant in zip(cal_nnls_ridge2(np.array(df_ridge[variants]) * np.expand_dims(df_sampled["resample_value"].values, 1),
                                                          np.array(df_ridge['frac']) * df_sampled["resample_value"].values,
                                                          0.5, len(char_vars)), char_vars):
                        current_ridge.append([variant, value, city, d, n])
    return current_ridge 

In [ ]:
ridge_res=[]
# city_ex = [cities_list[0]]#for testing

#with Pool(max_pool) as p:
#    global df_data2, df_data4, variants
#    pool_res = list(
#        tqdm(
#            p.imap(regress_city,
#                   cities_list),
#            total=len(cities_list)
#        )
#    )
#for res in pool_res:
#    ridge_res += res

for city in tqdm(cities_list, desc='Cities', position=0):
    ridge_res += regress_city(city)
            

In [ ]:
# load regression coefs to dataframe
df_ridge_res = pd.DataFrame(ridge_res)
df_ridge_res = df_ridge_res.rename(columns={0: 'variant', 1: 'weight', 2: 'location', 3: 'date', 4: 'iter'})
df_ridge_res['date'] = pd.to_datetime(df_ridge_res['date'])

df_final = df_ridge_res.sort_values(by=['date','variant']).reset_index(drop=True)
df_final

In [ ]:
df_ridge_res.to_csv('df_ridge_res.csv', index=False)

In [ ]:
df_prefinal = df_final.copy()

In [ ]:
df_final.dtypes

In [ ]:
df_final = df_prefinal.copy()

# TODO: have an actual classifier instead of mannually assigning presence 
df_final.loc[(df_final["variant"]=="B.1.1.529") & (df_final["date"]<np.datetime64("2021-11-01")),"weight"] = 0.0
df_final.loc[(df_final["variant"]=='omi-BA.2') & (df_final["date"]<np.datetime64("2021-11-01")),"weight"] = 0.0
#df_final.loc[(df_final["variant"]=="B.1.1.529") & (df_final["location"]!="Basel (catchment area ARA Basel)"),"weight"] = 0.0
# df_final.loc[(df_final["variant"]=="B.1.1.529") & (df_final["location"].isin(["Altenrhein (SG)"])),"weight"] = 0.0
df_final[(df_final['location'] == 'Basel (catchment area ARA Basel)') & (df_final['variant'] == 'B.1.1.529') & (df_final['date'] == '2021-10-15')]
# df_final = df_final[(df_final['date'] > '2021-11-14') & (df_final['date'] < '2022-01-02')]
# df_final = df_final[(df_final['date'] > '2021-11-14') & (df_final['date'] < '2021-12-02')]
df_final[(df_final['location'] == 'Basel (catchment area ARA Basel)') & (df_final['variant'] == 'B.1.1.529')]

df_final

# Smoothing

In [ ]:
#Smoothing
agg={}
agg2={}
agg3={}
df_smooth1={}
df_smooth2={}
d_test={}
#city_ex =[cities_list[0], cities_list[3]]
print(variants)
for city in tqdm(cities_list, desc='Cities', position=0):
    agg[city]={}
    agg2[city]={}
    agg3[city]={}
    df_smooth1[city]={}
    df_smooth2[city]={}
    d_test[city]={}

    for var in tqdm(variants, desc=city, position=1, leave=False):

        df_smooth1[city][var] = df_final[(df_final['location'] == city) & (df_final['variant'] == var)]

        df_smooth1[city][var] = df_smooth1[city][var].groupby(['variant', 'location', 'date'])['weight'].apply(list)

        df_smooth2[city][var] = df_smooth1[city][var].apply(pd.Series)

        agg2[city][var] = df_smooth2[city][var].reset_index()
        agg3[city][var] = (df_smooth2[city][var].apply(lambda x: lowess(x, np.arange(x.shape[0]).astype('float64'),
                                                            xvals = np.arange(x.shape[0]).astype('float64'),
                                                            frac= np.clip(20./df_smooth2[city][var].shape[0], 0, 2./3), it=0), 0)
                          ) if city != 'Kanton Basel' else (
                            # For KLZH data
                           df_smooth2[city][var].apply(lambda x: x.rolling(window=2, min_periods=1).mean(), 0)
                          )  

# Prevalence Plots

In [ ]:

# show all variants separately
sns.set_palette("dark")
# variants = sorted(variants, key=natsort_keygen())
for city in tqdm(cities_list, desc='Cities', position=0):

    #print(city)
    #fig = plt.figure()
    #fig, ax = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)
    #ax = [ax]

    for var in tqdm(variants, desc=city, position=1, leave=False):

        fig = plt.figure()
        fig, ax = plt.subplots(nrows=1, figsize=(12, 5), sharex=False)
        ax = [ax]

        # TODO be more clever with amplicons (for now we're just ignoring them)
        xvals = sorted(list(set(agg2[city][var]['date'])))


        sns.lineplot(x=xvals, y=np.clip(agg3[city][var].apply(np.nanmean, 1), 0., 1.),
                     ax=ax[0], markers=True,linewidth=3)

        ax[0].fill_between(xvals,
                           np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 5), 1).interpolate(), 0, 1),
                           np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 95), 1).interpolate(), 0, 1),
                           alpha=.4)



        
        ax[0].set_xlim((np.datetime64(start_date), np.datetime64(todaydate)))
    
        ax[0].set_ylabel(f"coeffs")
        ax[0].legend(loc="upper left")
        ax[0].set_title(f"{city}/{var}: smoothed regression curve")
        


#         plt.savefig(os.path.join(plots_dir, f'{city}_{var}.png')
        




In [ ]:
# show all variants at the same time
sns.set_palette("dark")
# variants = sorted(variants, key=natsort_keygen())

for city in tqdm(cities_list, desc='Cities', position=0):

    fig = plt.figure()
    fig, ax = plt.subplots(nrows=1, figsize=(20, 10), sharex=False)
    ax = [ax]
    l = 1
    for var in tqdm(variants, desc=city, position=1, leave=False):
        
#         if var == 'B.1.617.2':
#             l = 4
            
#         elif var == 'B.1.1.529':
#             l = 1
        
#         else:
#             l = 1

        # TODO be more clever with amplicons (for now we're just ignoring them)
        xvals = sorted(list(set(agg2[city][var]['date'])))
        


        sns.lineplot(x=xvals, y=np.clip(agg3[city][var].apply(np.nanmean, 1), 0., 1.),
                     ax=ax[0], markers=True, label=var, linewidth=l)

#         ax[0].fill_between(xvals,
#                            np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 5), 1).interpolate(), 0, 1),
#                            np.clip(agg3[city][var].apply(lambda x: np.percentile(x, 95), 1).interpolate(), 0, 1),
#                            alpha=.4)

        ax[0].set_xlim((np.datetime64(start_date), np.datetime64(todaydate)))
        ax[0].set_ylabel(f"coeffs")
#         ax[0].legend(loc="upper left")
        ax[0].set_title(f"{city}: smoothed regression curve")
    plt.savefig(os.path.join(plots_dir, f"{city.replace('/','-')}.png"))


# Prepare data for upload

## Load heatmap json file

In [ ]:
#from IPython.display import JSON

old_json=heatmaps_json_file
# print("reusing %s last modified: %s" % (old_json, time.ctime(os.path.getmtime(old_json))))
with open(old_json, 'r') as file:
     old_data = json.load(file)
#JSON(old_update_data) # only in Jupyter Lab

In [ ]:
old_data['undetermined']['Altenrhein (SG)'].keys()


In [ ]:
# variants


# Add Timeseries Data to Heatmap json File

In [ ]:
update_data={ }
tdf={city:{}  for city in cities_list}
tdf_mat={city:{}  for city in cities_list}

# HACK do not upload 'undetermined' for now
# variants = [v for v in variants if v != 'undetermined']
# variants.append('undetermined')

# this next line clips the plots from a given date
only_start_from={
    'Kanton Zürich':'2021-08-15', # start_date
#     'B.1.1.529':'2021-10-05',
}

for var in tqdm(variants_upload, desc='Variants', position=0):
    update_data[rename_variants.get(var, var)] = { }
    for city in tqdm(cities_list_upload, desc=var, position=1, leave=False):
        tdf[city][var] = agg3[city][var].apply(lambda x: {"proportion":np.clip(np.mean(x), 0., 1.),
                                                          "proportionLower":np.clip(np.percentile(x, 5), 0., 1.),
                                                          "proportionUpper":np.clip(np.percentile(x, 95), 0., 1.)},
                                               axis=1, result_type ='expand')
        if var in rename_variants:
            tdf[city][var].rename(index={var:rename_variants[var]},inplace=True)
        tdf[city][var] = tdf[city][var].reset_index()
        tdf[city][var]["date"] = tdf[city][var]["date"].astype("str")

        update_data[rename_variants.get(var, var)][city] = {
            #"updateDate": todaydate,
            "timeseriesSummary": [dict(tdf[city][var].iloc[i,]) for i in range(tdf[city][var].shape[0]) if ((city not in only_start_from) or (tdf[city][var].loc[i,'date'] >= only_start_from[city])) and ((var not in only_start_from) or (tdf[city][var].loc[i,'date'] >= only_start_from[var]))],
            "mutationOccurrences": (
                [
                    x for x in old_data[var][city]["mutationOccurrences"] if ((city not in only_start_from) or (x['date'] >= only_start_from[city])) and ((var not in only_start_from) or (x['date'] >= only_start_from[var]))
                ] # old_update_data[var][city]["mutationOccurrences"], #[dict(tdf_mat[city][var].iloc[i,]) for i in range(tdf_mat[city][var].shape[0])]
                if var != 'undetermined' else np.nan 
            ),
        }

import json
with open(update_data_combined_file, 'w') as file:
     file.write(json.dumps(update_data))

In [ ]:
# update_data['B.1.617.2']['Altenrhein (SG)']['mutationOccurrences']

In [ ]:
update_data['BA.2']['Altenrhein (SG)']['timeseriesSummary']

In [ ]:
update_data['undetermined']['Altenrhein (SG)'].keys()

In [ ]:
variants

In [ ]:
# update_data['B.1.1.529']['Kanton Basel']